<small><font color=gray>Notebook author: <a href="https://www.linkedin.com/in/olegmelnikov/" target="_blank">Oleg Melnikov</a> ©2021 onwards</font></small><hr style="margin:0;background-color:silver">

**[<font size=6>🧬Genomics</font>](https://www.kaggle.com/competitions/4dec23hse-genomics/rules)**. [**Instructions**](https://colab.research.google.com/drive/1owkYjuRGkx050LQnM3b3yTzd0Dr2XbeV) for running Colabs.

<small>**(Optional) CONSENT.** <mark>[ X ]</mark> We consent to sharing our Colab (after the assignment ends) with other students/instructors for educational purposes. We understand that sharing is optional and this decision will not affect our grade in any way. <font color=gray><i>(If ok with sharing your Colab for educational purposes, leave "X" in the check box.)</i></font></small>

In [ ]:
# from google.colab import drive; drive.mount('/content/drive')   # OK to enable, if your kaggle.json is stored in Google Drive

In [ ]:
# !pip -q install --upgrade --fo|                        # Kaggle dataset is copied as a single file and needs to be unzipped.
# !kaggle competitions leaderboard --show           # print public leaderboard

In [ ]:
# !nvidia-smi --query-gpu=gpu_name,memory.total,memory.free,memory.used --format=csv # GPU specs

In [ ]:
%%time
%%capture
%reset -f
!pip -q install -U sentence-transformers > log
from IPython.core.interactiveshell import InteractiveShell as IS; IS.ast_node_interactivity = "all"
import numpy as np, pandas as pd, time, matplotlib.pyplot as plt, os, plotly
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer as SBERT
from sklearn.svm import SVC, LinearSVC, NuSVC
ToCSV = lambda df, fname: df.round(2).to_csv(f'{fname}.csv', index_label='id') # rounds values to 2 decimals

class Timer():
  def __init__(self, lim:'RunTimeLimit'=60*5): self.t0, self.lim, _ = time.time(), lim, print(f'⏳ started. You have {lim} sec. Good luck!')
  def ShowTime(self):
    msg = f'Runtime is {time.time()-self.t0:.0f} sec'
    print(f'\033[91m\033[1m' + msg + f' > {self.lim} sec limit!!!\033[0m' if (time.time()-self.t0-1) > self.lim else msg)

np.set_printoptions(linewidth=10000, precision=4, edgeitems=20, suppress=True)
pd.set_option('display.max_rows', 100, 'display.max_columns', 100, 'display.max_colwidth', 100, 'display.precision', 2, 'display.max_rows', 4)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


CPU times: user 141 ms, sys: 30.6 ms, total: 171 ms
Wall time: 1.7 s


In [ ]:
vX = pd.read_csv('/Users/gena/Documents/4dec23hse-genomics/testX/testX.csv').set_index('id')
tYX = pd.read_csv('/Users/gena/Documents/4dec23hse-genomics/trainYX/trainYX.csv').set_index('id')
# vX

,DNA
id,
100000,TTGATTAATAAGATTCCTTGACACCCTTTGTAAAGTTTCTATTTCGTGTGAAATATCTATCTCTTCAAATCCTTTTAATTTATCTAGGTATTTGCT...
100001,ATTAGTAACGGAGGATTTACTAGATGTTTGGATTTATATTCTAATTTTATTCAGGTGGAAGGGATTGTTTTATGATTCAATAGTATACAGAGAATA...
...,...
119998,CGTCGGCATGCTCGGGCAGTGCGGCGGGCCAGCAGCGTGCCAGTTGTCGCGGGGCGGCCGGGCATCGCGGCGCCGGGCGGCAGCACTCCCGCGAAG...
119999,GCGAGGGCACGAAGGCACGACGGCAACGGCGGCGAGGAGCGCTGTGGCAACCGTCTCCGCGTTTGCGTGCGTACAGCCGAGAGCTGGTTCGCGCAG...


### EDA

In [ ]:
tmr = Timer() # runtime limit (in seconds). Add all of your code after the timer

❗Do not modify the setup above.

<hr color=red>

<font size=5>⏳</font> <strong><font color=orange size=5>Your Code, Documentation, Ideas and Timer - All Start Here...</font></strong>

Students: Keep all your definitions, code, documentation **between** ⏳ symbols.

## **Task 1. Preprocessing Pipeline**

Explain elements of your preprocessing pipeline i.e. feature engineering, subsampling, clustering, dimensionality reduction, etc.
1. Why did you choose these elements? (Something in EDA, prior experience,...? Btw, EDA is not required)
1. How do you evaluate the effectiveness of these elements?
1. What else have you tried that worked or didn't?

**Student's answer:**

In [ ]:
DNAs = vX["DNA"].values

In [ ]:
avg_seq_length, min_seq_length, max_seq_length = -1, 999999999999, -1
sum = 0
n = 0
greater_200 = 0
greater_300 = 0
greater_400 = 0
for dna in DNAs:
    dna_length = len(dna)
    if dna_length < min_seq_length:
        min_seq_length = dna_length
    if dna_length > max_seq_length:
        max_seq_length = dna_length
    sum += dna_length
    n += 1
    if dna_length >= 400:
        greater_400 += 1
    elif dna_length >= 300:
        greater_300 += 1
    elif dna_length >= 200:
        greater_200 += 1
print(sum/n)
print(min_seq_length)
print(max_seq_length)
print(greater_400)
print(greater_300)
print(greater_200)

268.43775
119
425
298
7383
7504


In [ ]:
tYX

,Y,DNA
id,,
0,0,CCTACTCTAAAGAGATTTGCAGTTGCGATGAGAAGCTTACCTCGAGAACCTAGTATGATGGAATTCATTGAAAGAACGTGATAAAAGTGAGATCCA...
1,1,CGGCGGTAGCAGCGCGGGCGAGACCCGCGAGGCGCGGCGGGTTGAACGCTTGCGGTCGACGGCGGGGCGTGCGTGTGCGACGAGCGCGCGCGCCGG...
...,...,...
99998,0,GATTCCTATTTCTCTGAAGAATTAGATAAGCACTCAGGTCAATTTCTGCATACTTATTAGTAGTCATTTGCCCGGGATGATAGACTGGTGGTGTTG...
99999,1,CTCGCGCCTTCGACGTTGCGGACTTGCCAGTCGGAAGTCGTCCCCTGGTGTGTCGCGCGACACAGCCCTTCCTTCGTTCACCCCACACTGGTAAGC...


In [ ]:
tYX_dnas = tYX.DNA.values

In [ ]:
len(tYX.DNA.values)

100000

In [ ]:
len(tYX_dnas)

100000

In [ ]:
len(tYX_dnas)

100000

In [ ]:
from tqdm import tqdm

In [ ]:
avg_seq_length, min_seq_length, max_seq_length = -1, 999999999999, -1
sum = 0
n = 0
greater_200 = 0
greater_300 = 0
greater_400 = 0
for dna in tqdm(tYX_dnas):
    dna_length = len(dna)
    if dna_length < min_seq_length:
        min_seq_length = dna_length
    if dna_length > max_seq_length:
        max_seq_length = dna_length
    sum += dna_length
    n += 1
    if dna_length >= 400:
        greater_400 += 1
    elif dna_length >= 300:
        greater_300 += 1
    elif dna_length >= 200:
        greater_200 += 1
print(sum/n)
print(min_seq_length)
print(max_seq_length)
print(greater_400)
print(greater_300)
print(greater_200)

100%|█████████████████████████████████████████████████████████████████████████████████| 100000/100000 [00:00<00:00, 1660222.85it/s]

268.07775
121
425
1563
36603
37629


We can see that in Baseline they were slicing the string up to first 50 values, howere the smallest seqence is 119 symbols long, this is the place from which we can get additional information. Baseline gives us accuracy of 0.9507
Lets see how we will improve with 119 symbols

Results of sequence change:
1) For some strange reason, 119 made it worse as well as 110 - down to 0.5 percent
2) However increaseing sequnce length to 100 made our accuracy score grow up to 0.9738

This gives us an ooportunity to add additional data entries in train from sequences with length > 200
We have 1563 sequences of length 400+
We have 36603 sequences of length 300+
We have 37629 sequences of length 200+
In total we have 1563 * 3 + 36603 * 2 + 37629 additional data entries in train which sum up to 115524 additional train entries.
This is more that +115% of our initial data

In [ ]:
# add aditional data entries
dna = tYX.DNA.values
targets = tYX.Y.values
new_targs, new_dnas = [], []
for target, dna_seq in zip(targets, dna):
    dna_seq_length = len(dna_seq)
    if dna_seq_length >= 400:
        first_seq = dna_seq[:100]
        second_seq = dna_seq[100:200]
        third_seq = dna_seq[200:300]
        fourth_seq = dna_seq[300:400]
        new_dnas += [first_seq, second_seq, third_seq, fourth_seq]
        new_targs += [target, target, target, target]
    elif dna_seq_length >= 300:
        first_seq = dna_seq[:100]
        second_seq = dna_seq[100:200]
        third_seq = dna_seq[200:300]
        new_dnas += [first_seq, second_seq, third_seq]
        new_targs += [target, target, target]
    elif dna_seq_length >= 200:
        first_seq = dna_seq[:100]
        second_seq = dna_seq[100:200]
        new_dnas += [first_seq, second_seq]
        new_targs += [target, target]
    else:
        first_seq = dna_seq[:100]
        new_dnas += [first_seq]
        new_targs += [target]

In [ ]:
new_tXY = pd.DataFrame({"Y":new_targs, "DNA":new_dnas})

In [ ]:
new_tXY

,Y,DNA
0,0,CCTACTCTAAAGAGATTTGCAGTTGCGATGAGAAGCTTACCTCGAGAACCTAGTATGATGGAATTCATTGAAAGAACGTGATAAAAGTGAGATCCA...
1,1,CGGCGGTAGCAGCGCGGGCGAGACCCGCGAGGCGCGGCGGGTTGAACGCTTGCGGTCGACGGCGGGGCGTGCGTGTGCGACGAGCGCGCGCGCCGG...
...,...,...
215522,1,GGGTTCGAGCGCTGCTTAATACGATAGTGCCATTGGATCTAAAAACACTTGCCGAGAGGCTCGGACATGGTTCCCGTCCGCCCGACAGTGCCACGC...
215523,1,CGTGGACTTGCCCAGTCGGAAGACGATCCCTGGTGGCCCCGCACCGCCCTTCCTTCGTGTCCACCACGCACGCGTAAGCGTGGCGGGTGCCGTCGC...


## **Task 2. Modeling Approach**
Explain your modeling approach, i.e. ideas you tried and why you thought they would be helpful.

1. How did these decisions guide you in modeling?
1. How do you evaluate the effectiveness of these elements?
1. What else have you tried that worked or didn't?

**Student's answer:**

[SBERT](https://www.sbert.net) generates 384-dimensional text embedding vectors for each text entry. See [more models](https://www.sbert.net/docs/pretrained_models.html).
* Only reputable publicly available embedding models are allowed (SBERT, USE, MUSE, LASER, ...). We want to prevent participants' training embeddings on test data.

In [ ]:
%%capture
sbert = SBERT('paraphrase-MiniLM-L6-v2')  # 4 seconds; loads SBERT embeddings model

In [ ]:
tEmb, vEmb = sbert.encode([s[:100] for s in new_tXY.DNA[:60000]]), sbert.encode([s[:100] for s in vX.DNA])  # embed all sequences to same-size vectors

In [ ]:
m = LinearSVC(random_state=0, max_iter=12000).fit(tEmb, new_tXY.Y[:60000])  # SVC is ok to use. See updated Rules.
m.score(tEmb, new_tXY.Y[:60000])   # in-sample accuracy

/Users/gena/Documents/venv/lib/python3.9/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


0.96925

In [ ]:
pY = pd.DataFrame(m.predict(vEmb), index=vX.index, columns=['y'])   # predicted targets
ToCSV((pY>0.5)*1, 'last_classic_ml_submission')

# **References:**

1. Remember to cite your sources here as well! At the least, your textbook should be cited. Google Scholar allows you to effortlessly copy/paste an APA citation format for books and publications. Also cite StackOverflow, package documentation, and other meaningful internet resources to help your peers learn from these (and to avoid plagiarism claims).
1. ...
1. ...

<font size=5>⌛</font> <strong><font color=orange size=5>Do not exceed competition's runtime limit!</font></strong>

<hr color=red>


In [ ]:
tmr.ShowTime()    # measure Colab's runtime. Do not remove. Keep as the last cell in your notebook.

Runtime is 43 sec


# 💡**Starter Ideas**

1. Learn about DNA [&#127910;](https://www.youtube.com/results?search_query=nucleotides+genes+amino+acids+)
1. Try a larger training sample.
1. Try longer training DNA strings, but SBERT may have a cap on string length, so you might split DNA into several strings and then concatenate or average resulting vectors
1. Try other pretrained SBERT models. Note that DNA sequence uses ACGT letters, but many other models were trained on multilingual text. So, you might prefer those that were trained on mostly ASCII.
1. SBERT is trained on word tokens (typically, separated by spaces), but DNA sequence has no spaces. Try placing spaces after every character or some semantically meaningful subsequences (this might require more domain knowledge).
1. Try Google's [USE](https://tfhub.dev/google/universal-sentence-encoder-multilingual/3) embedding models
1. Try Facebook's [LASER](https://github.com/facebookresearch/LASER) and [others](https://tfhub.dev/s?module-type=text-language-model).
1. Try [Enformer](https://tfhub.dev/deepmind/enformer/1) for gene expressions. See [DeepMind paper](https://deepmind.com/blog/article/enformer).
1. Try building your own embeddings on the given sequences. SBERT and other packages make it easy (just a few lines), but it may take too much time.
1. Assess distribution of character patterns (single, doubles, triplets, ...). For example, an ACGT string generates AC, CG, GT doubles and ACG and CGT triplets. Does one class have more subsequences of some type? This might be a feature in your model.
1. Try features built as counts of subsequences (singles, doubles, triplets, ...). Consider EDA first.
1. Concatenate or otherwise combine multiple embeddings derived from each gene string
1. Learn from [*The genetic code*](https://www.khanacademy.org/science/ap-biology/gene-expression-and-regulation/translation/a/the-genetic-code-discovery-and-properties), Khan Academy.
1. Learn from [*Apply Machine Learning Algorithms for Genomics Data Classification*](https://medium.com/mlearning-ai/apply-machine-learning-algorithms-for-genomics-data-classification-132972933723)
1. Learn from [*Efficient counting of k-mers in DNA sequences using a bloom filter*](https://bmcbioinformatics.biomedcentral.com/articles/10.1186/1471-2105-12-333) Páll Melsted et al. 2011
1. Try [Byte Pair Encoding](https://www.derczynski.com/papers/archive/BPE_Gage.pdf) and [SentencePiece](https://arxiv.org/pdf/1808.06226.pdf) to auto identification of "important" [k-mers](https://en.wikipedia.org/wiki/K-mer) (substrings)